# Libraries

In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

import pandas, scipy, numpy, matplotlib, seaborn

# 1. Import Data and EDA

In [0]:
# 1. Read the data already in the catalog:
data = spark.read.table("main.default.mental_health_care_in_the_last_4_weeks")
display(data)

In [0]:
# 2. Initial analysis of the data:
print(f"Number of columns: {len(data.columns)}")
print(f"Columns: {data.columns}")


# 3. Check for missing values
missing_df = data.select([
    sum(col(c)
        .isNull().cast("int")
        )
        .alias(c) 
        for c in data.columns
])

display(missing_df)





In [0]:
data.printSchema()

# Data Visualization via Tabluae

In [0]:
%sql
-- Create a clean view for your visualizations
CREATE OR REPLACE VIEW main.default.vw_mental_health_clean AS
SELECT 
    Indicator,
    `Group`,
    State,
    -- Convert the 'Value' column to a double so Tableau can sum/average it
    CAST(Value AS DOUBLE) AS Mental_Health_Value,
    -- Capture the confidence intervals if you're doing statistical analysis
    CAST(`LowCI` AS DOUBLE) AS Low_Confidence_Interval,
    CAST(`HighCI` AS DOUBLE) AS High_Confidence_Interval,
    `Time Period`,
    `Time Period Label`
FROM main.default.mental_health_care_in_the_last_4_weeks
WHERE `Group` = 'By State' 
  AND Value IS NOT NULL;

In [0]:

%sql
SELECT * FROM main.default.vw_mental_health_clean LIMIT 10;

In [0]:
%sql
-- SELECT the 10 states with the highest mental health scores
SELECT 
          AVG(Mental_Health_Value) AS avg_mental_health_value
          , State
FROM main.default.vw_mental_health_clean
GROUP BY State
ORDER BY avg_mental_health_value DESC
LIMIT 10;


Databricks visualization. Run in Databricks to view.